In [1]:
import numpy as np
import pregel
import pandas as pd

In [4]:
# Read the Excel file
payments_df = pd.read_excel('Payments.xlsx')
bad_nodes_df = pd.read_excel('bad_sender.xlsx')


# Extract columns into separate lists
sender_list = payments_df['Sender'].astype(int).tolist()
receiver_list = payments_df['Receiver'].astype(int).tolist()
amount_list = payments_df['Amount'].astype(int).tolist()
bad_sender_list = bad_nodes_df['Bad Sender'].astype(int).tolist()

In [5]:
nodes_list = list(set(sender_list + receiver_list + bad_sender_list))
outgoing_edges = {}

# Replaced multiple directed edges from a node to another anoder node with single directed edge with amounts added
for sender, receiver, amount in zip(sender_list, receiver_list, amount_list):
    if sender not in outgoing_edges:
        outgoing_edges[sender] = []
    found = False
    for i, (recv, amt) in enumerate(outgoing_edges[sender]):
        if recv == receiver:
            outgoing_edges[sender][i] = (recv, amt + amount)
            found = True
            break
    if not found:
        outgoing_edges[sender].append((receiver, amount))
        
len(bad_sender_list)

20

In [26]:
class TrustRankVertex(pregel.Vertex):
    
    def __init__(self, vertex_id, bad_sender_list, outgoing_edges, vertex_objs, damping_factor=0.85, max_iterations=50):
        
        out_vertices = []
        if vertex_id in outgoing_edges:
            out_vertices = outgoing_edges[vertex_id]
        
        # print(len(out_vertices))
        super().__init__(vertex_id, 0, list(out_vertices))

        self.damping_factor = damping_factor
        self.max_iterations = max_iterations
        if vertex_id in outgoing_edges:
            self.outgoing_edges = outgoing_edges[vertex_id]
        else:
            self.outgoing_edges = []

        if vertex_id in bad_sender_list:
            self.bad_node = True
            self.value = 1/len(bad_sender_list)
        else:
            self.bad_node = False
            self.value = 0

        self.out_deg = 0

        self.out_deg = sum(amount for _, amount in out_vertices)
            
        self.const_value = self.value
        self.vertex_objs = vertex_objs

    def update(self):
        if self.superstep < self.max_iterations:
            
            self.value = (1-self.damping_factor) * self.const_value + self.damping_factor* sum([trustrank for (vertex,trustrank) in self.incoming_messages])
            if self.out_deg != 0:
                # self.outgoing_messages = [(id, self.value * amount/ max(1, self.out_deg)) for id, amount in self.outgoing_edges]
                self.outgoing_messages = [(self.vertex_objs[id], self.value * amount/ self.out_deg) for id, amount in self.out_vertices]
            else:
                self.outgoing_messages = []
        else:
            # stop after 50 supersteps
            self.active = False

In [27]:
vertices = []
vertex_objs = {}
for node_id in nodes_list:
    if node_id in vertex_objs:
        vertices.append(vertex_objs[node_id])
    else:
        vertex_objs[node_id] = TrustRankVertex(node_id, bad_sender_list, outgoing_edges, vertex_objs, 0.85, 50)
        vertices.append(vertex_objs[node_id])

In [28]:
Pregel = pregel.Pregel(vertices, 4)
Pregel.run()
for vertex in Pregel.vertices:
    print(vertex.id, vertex.value)

2050 0.0
2052 0.0
2053 0.0
2054 0.0
2057 0.0
2060 0.0
2061 0.0
2062 0.0
2066 0.0
2071 0.0
2072 0.0
2073 0.0
2075 0.0
2076 0.0
2078 0.0
2080 0.0
2081 0.0
2084 0.0
2085 0.0
2088 0.0
2089 0.0
2090 0.0
2093 0.0
2097 0.0
2100 0.0
2101 0.0
2104 0.0
2107 0.0
2108 0.0
2109 0.0
2111 0.0
2124 0.0
2126 0.0
2127 0.0
2136 0.0
2137 0.0
2142 0.0
2145 0.0
2146 0.0
2150 0.0
2151 0.0
2152 0.0
2158 0.0
2162 0.0
2164 0.0
2166 0.0
2168 0.0
2169 0.0
2170 0.0
2171 0.0
2172 0.0
2174 0.0
2177 0.0
2182 0.0
2184 0.0
2188 0.0
2189 0.0
2190 0.0
1001 0.0014958162735442745
1002 0.0001183049489287393
1003 0.0002616960104011364
1004 0.0027876223282081915
1005 0.0065028619979916324
1006 0.0006341586853778971
1007 0.028530440071183258
1008 0.0015593746867149112
1009 0.0
1010 0.0003339316042213136
1011 0.008897648445719832
1012 0.0
1013 0.008565816567545646
1014 0.0022187552836970726
1015 0.0
1016 0.00541792242186249
1017 0.0
1018 0.0
1019 2.7583012569325782e-05
1020 0.0010377902730841371
1021 0.005162885003394185
1022 0